In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import pandas as pd
import PreClf, MultiPreClf
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [5]:
# MNIST 
X_, y_ = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")
# Preprocessing normalisation 
X_ = X_ / 255

y_ = y_.astype(int)

In [6]:
# On-hot encoding
encoder = OneHotEncoder(sparse=False)
y_one_hot = encoder.fit_transform(y_.reshape(-1, 1))



m:\ML\ML_regs\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_one_hot, test_size=0.2, random_state=42)


In [8]:
model = MultiPreClf.MultilayerPerceptron(alpha=0.01, 
                                n_iterations=2, 
                                input_layer_size=784, 
                                hidden_layers_size=[32, 16], 
                                n_outputs=10)

Weights shape: (784, 32)
Weights shape: (32, 16)
Weights shape: (16, 10)


In [9]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# model.train(X_train, y_train, X_test, y_test, batch_size=8)

activations = model.forward_propagation(X_train[0:8])

A 0 : (8, 784) 
W 1 : (784, 32) 
Z 1 - Linear output shape: (8, 32)
A 1 - Activation(relu) output shape: (8, 32)
W 2 : (32, 16) 
Z 2 - Linear output shape: (8, 16)
A 2 - Activation(relu) output shape: (8, 16)
Z 2 - Linear output shape: (8, 16) 
W 2 - Weights shape: (16, 10) 
Z 3 - Linear output shape: (8, 10)
A 3 - Activation (softmax) output shape: (8, 10)


In [37]:
model.back_propagation(X_train[0:8], y_train[0:8], activations)

-1
 δl(8, 10) =  dL[3]/dA3 (8, 10) * dA3/dZ3 (8, 10)
A-2 (16, 8).T * δl (8, 10)
W[-1] (16, 10)
-2
A-3 (32, 8).T * δl (8, 16)
W[-2] (32, 16)
-3
A-4 (784, 8).T * δl (8, 32)
W[-3] (784, 32)


In [225]:
for layer in range(len(model.weights) ):
    print(layer+1, model.weights[layer].shape)

1 (784, 32)
2 (32, 16)
3 (16, 10)


In [227]:
for activation in range(len(activations)):
    print(activation, activations[activation].shape)

0 (8, 784)
1 (8, 32)
2 (8, 16)
3 (8, 10)


In [234]:
# loop from back of the weights, 0 because 
for layer in range(len(model.weights)-1, 0, -1):
    print(layer, model.weights[layer].shape, activations[layer].shape)

2 (16, 10) (8, 16)
1 (32, 16) (8, 32)


In [19]:
len(model.weights) + 1


4

In [17]:
for l in range(1, len(model.weights) + 1):
    print(-l, model.weights[-l].shape, activations[-l].shape)

-1 (16, 10) (8, 10)
-2 (32, 16) (8, 16)
-3 (784, 32) (8, 32)


In [26]:
model.weights[-3].shape

(784, 32)

In [33]:
-2 + 1

-1